# CHAPTER 4.  Left Outer Join

# Join 의 종류

![](sparkr_04_01.jpg)
![](sparkr_04_02.jpg)
![](sparkr_04_03.jpg)

## Join의 예제
![](sparkr_04_04.jpg)
![](sparkr_04_05.jpg)
![](sparkr_04_06.jpg)

## Implementation of Left Outer Join in MapReduce

앞의 예제 Query3에서 보여준 것 같이, 팔린 상품이 얼마만큼의 지역에서 팔려는지 알아보는 것을 2단계를 거침.

- 1단계 : 앞의 Query1 과 같이, 모든 팔린 상품과 지역ID을 찾음.
- 2단계 : 앞의 Query3의 결과를 보여주기 위해서 유니크한 지역ID 개수를 셈.


![](sparkr_04_07.jpg)
![](sparkr_04_08.jpg)

### PySpark로 구현

#### Step 3: Create a Spark context object

In [1]:
from pyspark import SparkContext
sc = SparkContext() 
sc

#### Step 4: Create an RDD for the users data

In [8]:
users = sc.textFile('users.txt', 1);
users.collect()

[u'u1,UT', u'u2,GA', u'u3,CA', u'u4,CA', u'u5,GA']

#### Step 5: Create usersRDD (the right table)

In [12]:
def make_user_rdd( line ) :
    userRecord = line.split(",")
    userID = userRecord[0]
    location = userRecord[1]
    result = []
    result.append( (userID, location) )
    return result 

s = 'u1,UT'
make_user_rdd( s )

[('u1', 'UT')]

In [13]:
usersRDD = users.flatMap( make_user_rdd )

In [14]:
usersRDD.collect()

[(u'u1', u'UT'),
 (u'u2', u'GA'),
 (u'u3', u'CA'),
 (u'u4', u'CA'),
 (u'u5', u'GA')]

#### Step 6: Create an RDD for the transactions data

In [16]:
transactions = sc.textFile('transactions.txt', 1);
transactions.collect()

[u't1,p3,u1,3,330',
 u't2,p1,u2,1,400',
 u't3,p1,u1,3,600',
 u't4,p2,u2,10,1000',
 u't5,p4,u4,9,90',
 u't6,p1,u1,4,120',
 u't7,p4,u1,8,160',
 u't8,p4,u5,2,40']

In [18]:
def make_tran_rdd( line ) :
    transactionRecord = line.split(",");
    userID = transactionRecord[2];
    product = transactionRecord[1];
    result = []
    result.append( (userID, product) )
    return result 

s = u't1,p3,u1,3,330'
make_tran_rdd(s)

[(u'u1', u'p3')]

In [19]:
transactionsRDD = transactions.flatMap( make_tran_rdd )
transactionsRDD.collect()

[(u'u1', u'p3'),
 (u'u2', u'p1'),
 (u'u1', u'p1'),
 (u'u2', u'p2'),
 (u'u4', u'p4'),
 (u'u1', u'p1'),
 (u'u1', u'p4'),
 (u'u5', u'p4')]

#### Step 8: Use Spark’s built-in JavaPairRDD.leftOuterJoin() method

In [21]:
joined = transactionsRDD.leftOuterJoin(usersRDD);
joined.collect()

[(u'u5', (u'p4', u'GA')),
 (u'u1', (u'p3', u'UT')),
 (u'u1', (u'p1', u'UT')),
 (u'u1', (u'p1', u'UT')),
 (u'u1', (u'p4', u'UT')),
 (u'u4', (u'p4', u'CA')),
 (u'u2', (u'p1', u'GA')),
 (u'u2', (u'p2', u'GA'))]

#### Step 9: Create (product, location) pairs

In [25]:
def make_pairs( t ) :
    value = t[1]
    result = []
    result.append( (value[0], value[1]) ) 
    return result 

In [26]:
products = joined.flatMap( make_pairs )
products.collect()

[(u'p4', u'GA'),
 (u'p3', u'UT'),
 (u'p1', u'UT'),
 (u'p1', u'UT'),
 (u'p4', u'UT'),
 (u'p4', u'CA'),
 (u'p1', u'GA'),
 (u'p2', u'GA')]

#### Step 10: Group (K=product, V=location) pairs by key

In [29]:
productByLocations = products.groupByKey();
debug2 = productByLocations.collect()
for t2 in  debug2 :
    print "debug2 key={}\t value={}".format( t2[0],  ",".join([str(x) for x in t2[1]] )   )

debug2 key=p2	 value=GA
debug2 key=p4	 value=GA,UT,CA
debug2 key=p3	 value=UT
debug2 key=p1	 value=UT,UT,GA


#### Step 11: Create final output (K=product, V=Set(location))

In [30]:
def make_pairs_by_key( s ) :
    uniqueLocations = {} # HashMap
    for location in s :
        uniqueLocations[ location ] = 1
        
    return ( uniqueLocations, len(uniqueLocations)  )

In [32]:
productByUniqueLocations = productByLocations.mapValues( make_pairs_by_key )

productByUniqueLocations.collect()

[(u'p2', ({u'GA': 1}, 1)),
 (u'p4', ({u'CA': 1, u'GA': 1, u'UT': 1}, 3)),
 (u'p3', ({u'UT': 1}, 1)),
 (u'p1', ({u'GA': 1, u'UT': 1}, 2))]

### SparkR의 dataframe을 구현

In [1]:
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))

sc <- sparkR.init(master="local[*]", sparkPackages="com.databricks:spark-csv_2.10:1.4.0" )


Attaching package: ‘SparkR’

The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var

The following objects are masked from ‘package:base’:

    colnames, colnames<-, intersect, rank, rbind, sample, subset,
    summary, table, transform



Launching java with spark-submit command /usr/local/spark/bin/spark-submit  --packages com.databricks:spark-csv_2.10:1.4.0 sparkr-shell /tmp/RtmpKL9K1z/backend_porta210fb3dd3 


In [2]:
sqlContext <- sparkRSQL.init(sc)

In [27]:
user_df <- read.df(sqlContext, 
                            'users.csv', 
                            header='true', 
                            source = "com.databricks.spark.csv", 
                            inferSchema='true')
head(user_df, n=10)

,user,region
1,u1,UT
2,u2,GA
3,u3,CA
4,u4,CA
5,u5,GA


In [26]:
trans_df <- read.df(sqlContext, 
                            'transactions.csv', 
                            header='true', 
                            source = "com.databricks.spark.csv", 
                            inferSchema='true')
head(trans_df, n=20)

,tr,product,user,cnt,price
1,t1,p3,u1,3,330
2,t2,p1,u2,1,400
3,t3,p1,u1,3,600
4,t4,p2,u2,10,1000
5,t5,p4,u4,9,90
6,t6,p1,u1,4,120
7,t7,p4,u1,8,160
8,t8,p4,u5,2,40


- join() 함수 : https://spark.apache.org/docs/1.4.0/api/R/join.html

Description

Join two DataFrames based on the given join expression.

Usage

#### S4 method for signature 'DataFrame,DataFrame'
join(x, y, joinExpr = NULL, joinType = NULL)
Arguments

x	
A Spark DataFrame

y	
A Spark DataFrame

joinExpr	
(Optional) The expression used to perform the join. joinExpr must be a Column expression. If joinExpr is omitted, join() wil perform a Cartesian join

joinType	
The type of join to perform. The following join types are available: 'inner', 'outer', 'left_outer', 'right_outer', 'semijoin'. The default joinType is "inner".

In [23]:
inner_join_df <- join(user_df, trans_df, user_df$user == trans_df$user, 'inner')
head( inner_join_df , 20)

,user,region,tr,product,user,cnt,price
1,u1,UT,t1,p3,u1,3,330
2,u1,UT,t3,p1,u1,3,600
3,u1,UT,t6,p1,u1,4,120
4,u1,UT,t7,p4,u1,8,160
5,u2,GA,t2,p1,u2,1,400
6,u2,GA,t4,p2,u2,10,1000
7,u4,CA,t5,p4,u4,9,90
8,u5,GA,t8,p4,u5,2,40


In [24]:
right_join_df <- join(user_df, trans_df, user_df$user == trans_df$user, 'right_outer')
head( right_join_df  , 20 )

,user,region,tr,product,user,cnt,price
1,u1,UT,t1,p3,u1,3,330
2,u1,UT,t3,p1,u1,3,600
3,u1,UT,t6,p1,u1,4,120
4,u1,UT,t7,p4,u1,8,160
5,u2,GA,t2,p1,u2,1,400
6,u2,GA,t4,p2,u2,10,1000
7,u4,CA,t5,p4,u4,9,90
8,u5,GA,t8,p4,u5,2,40


In [25]:
left_join_df <- join(user_df, trans_df, user_df$user == trans_df$user, 'left_outer')
head( left_join_df  , 20 )

,user,region,tr,product,user,cnt,price
1,u1,UT,t1,p3,u1,3,330
2,u1,UT,t3,p1,u1,3,600
3,u1,UT,t6,p1,u1,4,120
4,u1,UT,t7,p4,u1,8,160
5,u2,GA,t2,p1,u2,1,400
6,u2,GA,t4,p2,u2,10,1000
7,u3,CA,NA,NA,NA,NA,NA
8,u4,CA,t5,p4,u4,9,90
9,u5,GA,t8,p4,u5,2,40


In [41]:
region_counts <- summarize(groupBy(right_join_df, right_join_df$product), count = n( right_join_df$region ))
head(region_counts)

,product,count
1,p1,3
2,p2,1
3,p3,1
4,p4,3
